# Pawpularity Contest with Swin Transformers
![](https://storage.googleapis.com/kaggle-media/competitions/Petfinder/PetFinder%20-%20Logo.png)

![](https://www.petfinder.my/images/cuteness_meter-showcase.jpg)


## Introduction

A picture is worth a thousand words. But did you know a picture can save a thousand lives? Millions of stray animals suffer on the streets or are euthanized in shelters every day around the world. You might expect pets with attractive photos to generate more interest and be adopted faster. But what makes a good picture? With the help of data science, you may be able to accurately determine a pet photo's appeal and even suggest improvements to give these rescue animals a higher chance of loving homes.

[PetFinder.my](https://petfinder.my/) is Malaysia's leading animal welfare platform, featuring over 180,000 animals with 54,000 happily adopted. PetFinder collaborates closely with animal lovers, media, corporations, and global organizations to improve animal welfare.

Currently, PetFinder.my uses a basic [Cuteness Meter](https://petfinder.my/cutenessmeter) to rank pet photos. It analyzes picture composition and other factors compared to the performance of thousands of pet profiles. While this basic tool is helpful, it's still in an experimental stage and the algorithm could be improved.

In this competition, you'll analyze raw images and metadata to predict the “Pawpularity” of pet photos. You'll train and test your model on PetFinder.my's thousands of pet profiles.

![](https://www.petfinder.my/images/cuteness_meter.jpg)

Image Source: [Petfinder.my](https://www.petfinder.my/images/cuteness_meter.jpg)

## Table of Contents
---
- [Introduction](#Introduction)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
- [Data Cleaning](#Data-Cleaning)
- [Normalization and Standardization](#Normalization-and-Standardization)
- [Data Visualization](#Data-Visualization)
- [Model Preparation](#Model-Preparation)
- [Model Results](#Model-Results)
  - [Linear Regression](#Linear-Regression)
  - [KNN](#KNN)
  - [Random Forest](#Random-Forest)
  - [Support Vector Machine](#Support-Vector-Machine)
  - [Gradient Boosting](#Gradient-Boosting)
- [Best Performing Model](#Best-Performing-Model)  
- [Partial Dependence Plots](#Partial-Dependence-Plots)
- [Discussion and Recommendations](#Discussion-and-Recommendations)

Setup Kaggle dataset.

In [25]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"axion004","key":"aea22e38d52b3227012bd30563c22cae"}'}

In [26]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Apr 24 03:28 kaggle.json


In [27]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [28]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [29]:
# List available datasets.
!kaggle datasets list

ref                                                  title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rahulvyasm/netflix-movies-and-tv-shows               Netflix Movies and TV Shows                      1MB  2024-04-10 09:48:38           4833        100  1.0              
sudarshan24byte/online-food-dataset                  Online Food Dataset                              3KB  2024-03-02 18:50:30          31725        604  0.9411765        
fatemehmehrparvar/obesity-levels                     Obesity Levels                                  58KB  2024-04-07 16:28:30           3890         60  0.88235295       
asaniczka/university-employee-salaries-2011-present  University Employee Salaries (2011 - Present)   17MB  2024-04-07 10:11:15           197

In [30]:
! kaggle competitions download -c 'petfinder-pawpularity-score'

 99% 969M/983M [00:12<00:00, 130MB/s]
100% 983M/983M [00:12<00:00, 82.5MB/s]


In [33]:
# Unzip the training and test directories
! mkdir train
! mkdir test
! unzip petfinder-pawpularity-score.zip -d petfinder-pawpularity-score

Streaming output truncated to the last 5000 lines.
  inflating: petfinder-pawpularity-score/train/7e3d6d03218db540c6125630572aa7cd.jpg  
  inflating: petfinder-pawpularity-score/train/7e51d9305ec464c1f77cfee35ceb1b4f.jpg  
  inflating: petfinder-pawpularity-score/train/7e56aa71d1e091e004703283323e8e7f.jpg  
  inflating: petfinder-pawpularity-score/train/7e5d1650d5cda4d68f2ee8ce17c29b25.jpg  
  inflating: petfinder-pawpularity-score/train/7e601feb12f2fb03c2d342e54cd47b1e.jpg  
  inflating: petfinder-pawpularity-score/train/7e6ca633d2a751af58e1cc220ca519b3.jpg  
  inflating: petfinder-pawpularity-score/train/7e717923c22053e51861a24aae701fb0.jpg  
  inflating: petfinder-pawpularity-score/train/7e761f47cc1e3038a431f9f196234ab9.jpg  
  inflating: petfinder-pawpularity-score/train/7e7921ed945c3c863882340360c9157d.jpg  
  inflating: petfinder-pawpularity-score/train/7e8763e28a01c059b7a8d119c5ae0dc0.jpg  
  inflating: petfinder-pawpularity-score/train/7e93691bef9bc381590cf004358ee11a.jpg  
  i

In [6]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [34]:
from pathlib import Path
dataset_path = Path('./petfinder-pawpularity-score/')

In [35]:
N_FOLDS = 10
FOLDS = range(N_FOLDS)
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE
GRADIENT_ACCUMULATION = 1
IMAGE_SIZE = 384
BACKBONE = 'swin_large_patch4_window12_384'
MAX_LR = 2e-5
DECAY = 0.1
EPOCHS = 5
WORKERS = 2
VERBOSE = None
MIXUP = 0.3
NUM_TARGET = 50

IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)

We start by importing the necessary libraries.

In [36]:
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
import json
from tqdm import tqdm
import random
import time
import gc

from scipy.special import expit, logit, softmax
from sklearn.model_selection import KFold
from skimage.transform import rescale, resize, downscale_local_mean
import cv2
import albumentations

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.cuda.amp import autocast, GradScaler

In [37]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [38]:
def seed_torch(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
    if torch.backends.cudnn.is_available:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [39]:
valid_transforms = albumentations.Compose([
    albumentations.CenterCrop(IMAGE_SIZE, IMAGE_SIZE, always_apply=True, p=1),
    albumentations.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=1., always_apply=True, p=1),
])


In [40]:
def load_image(path):
    #path = f'../input/petfinder-pawpularity-score/train/{id}.jpg'
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

class PetDataset(Dataset):
    def __init__(self,
                 data,
                 image_size,
                 target=True,
                 aug=None,
                 channel_first=True,
                ):
        super(PetDataset, self).__init__()
        self.data = data
        self.image_size = image_size
        self.target = target
        self.aug = aug
        self.channel_first = channel_first

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = load_image(self.data.path[idx])
        image = image.astype('float32')
        image /= 255.
        h, w, c = image.shape
        if h > w:
            image = resize(image, (int(round(self.image_size * h / w)), self.image_size), preserve_range=True, anti_aliasing=True)
        else:
            image = resize(image, (self.image_size, int(round(self.image_size * w / h))), preserve_range=True, anti_aliasing=True)
        if self.aug is not None:
            this = self.aug(image=image)
            image = this['image']
        if self.channel_first:
            image = image.transpose(2, 0, 1)
        target = 0
        binary_target = 0
        if self.target:
            target = self.data.norm_score[idx]
            binary_target = int(round(target * NUM_TARGET))
        return {'image':torch.tensor(image, dtype=torch.half),
                'target':torch.tensor([target], dtype=torch.half),
                'binary_target':torch.tensor([binary_target], dtype=torch.long),
               }

In [41]:
test_df = pd.read_csv('test.csv')
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,1,petfinder-pawpularity-score/test/4128bae221838...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,1,petfinder-pawpularity-score/test/43a2262d7738e...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,1,petfinder-pawpularity-score/test/4e429cead1848...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,1,petfinder-pawpularity-score/test/80bc3ccafcc51...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,1,petfinder-pawpularity-score/test/8f49844c38293...
5,b03f7041962238a7c9d6537e22f9b017,0,0,1,1,1,1,1,1,1,0,1,0,1,petfinder-pawpularity-score/test/b03f704196223...
6,c978013571258ed6d4637f6e8cc9d6a3,1,0,0,0,1,1,0,1,0,1,1,1,1,petfinder-pawpularity-score/test/c978013571258...
7,e0de453c1bffc20c22b072b34b54e50f,1,0,1,0,0,0,0,0,1,0,0,1,1,petfinder-pawpularity-score/test/e0de453c1bffc...


In [42]:
dataset = PetDataset(test_df, IMAGE_SIZE, aug=valid_transforms, target=False, channel_first=False)
#plt.imshow(dataset.__getitem__(1)['image'].data)

In [43]:
criterion = nn.BCEWithLogitsLoss()

class PetModel(nn.Module):
    def __init__(self, backbone_name, pretrained=True, loss=False):
        super(PetModel, self).__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=NUM_TARGET, drop_rate=0.00, drop_path_rate=0.00)
        self.loss = loss

    def forward(self, input_dict, mixup=0):
        x = input_dict['image']
        if mixup:
            batch_size = x.size(0)
            perm = torch.randperm(batch_size).to(x.device)
            x_perm = x[perm]
            beta = torch.distributions.beta.Beta(torch.tensor([0.5]), torch.tensor([0.5]))
            alpha = beta.sample(torch.tensor([batch_size])).to(x.device)
            alpha = alpha.view((alpha.shape[0], 1, 1, 1))
            x = alpha*x + (1 - alpha)*x_perm

        x = self.backbone(x)
        pred = torch.sigmoid(x).sum(1) / NUM_TARGET

        loss = None
        if self.loss:
            #target = input_dict['target']
            binary_target = input_dict['binary_target']
            #binary_target = 1 - np.eye(num_class)[target].cumsum(1)[:, :-1]
            binary_target = 1. - F.one_hot(binary_target, NUM_TARGET+1).cumsum(-1)[:, :, :-1].transpose(1, 2).squeeze(-1)
            #print(binary_target.shape, x.shape)
            if mixup:
                target_perm = binary_target[perm]
                alpha = alpha.view((alpha.shape[0], 1))
                target = alpha*binary_target + (1 - alpha)*target_perm
            loss = criterion(x, binary_target)
        return {
            'pred' : pred,
            'loss' : loss,
        }

In [44]:
def test_epoch(loader, model, device):

    model.eval()
    LOGITS = []

    with torch.no_grad():
        if 1:
            if VERBOSE is not None:
                bar = tqdm(range(len(loader)))
            else:
                bar = range(len(loader))
            load_iter = iter(loader)
            batch = load_iter.next()
            batch = {k:batch[k].to(device, non_blocking=True) for k in batch.keys() }

            for i in bar:
                input_dict = batch.copy()
                if i + 1 < len(loader):
                    batch = load_iter.next()
                    batch = {k:batch[k].to(device, non_blocking=True) for k in batch.keys() }

                out_dict = model(input_dict)
                pred = out_dict['pred']
                LOGITS.append(pred.detach())


    LOGITS = torch.cat(LOGITS).cpu().numpy().ravel()

    return LOGITS

def load_checkpoint(fold, seed, dirname, fname, test=False):
    model = PetModel(BACKBONE, pretrained=(not test), loss=True).to(device)
    checkpoint = torch.load('../input/%s/%s_%d_%d.pt' % (dirname, fname, fold, seed))
    model.load_state_dict(checkpoint['model'])
    model = model.half()
    return model

VERBOSE = None

In [45]:
preds = []


In [46]:
fname = 'pet_052'
print(fname)
#FOLDS = [1,2,3,4]
device = torch.device('cuda')
test_dataset = PetDataset(test_df, IMAGE_SIZE, target=False, aug=valid_transforms)
for seed in [0]:
    for fold in tqdm(FOLDS):
        seed_torch(seed)


        test_data_loader = DataLoader(
            test_dataset,
            batch_size=VALID_BATCH_SIZE,
            num_workers=WORKERS,
            shuffle=False,
            #pin_memory=True,
        )
        model = load_checkpoint(fold, seed, 'pet-models',  fname, test=True)
        pred = test_epoch(test_data_loader, model, device) * 100
        preds.append(pred)
        del test_data_loader, model
        gc.collect()

pet_052


  0%|          | 0/10 [00:05<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../input/pet-models/pet_052_0_0.pt'

In [ ]:
fname = 'pet_064'
print(fname)
#FOLDS = [1,2,3,4]
device = torch.device('cuda')
for seed in [0]:
    for fold in tqdm(FOLDS):
        seed_torch(seed)

        test_data_loader = DataLoader(
            test_dataset,
            batch_size=VALID_BATCH_SIZE,
            num_workers=WORKERS,
            shuffle=False,
            #pin_memory=True,
        )
        model = load_checkpoint(fold, seed, 'pet64model',  fname, test=True)
        pred = test_epoch(test_data_loader, model, device) * 100
        preds.append(pred)
        del test_data_loader, model
        gc.collect()

pet_064


100%|██████████| 10/10 [02:04<00:00, 12.47s/it]


In [ ]:
preds = np.mean(preds, axis=0)
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df['Pawpularity'] = preds
sample_df.to_csv('submission.csv',index=False)


In [ ]:
sample_df

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,40.18750
1,43a2262d7738e3d420d453815151079e,40.28125
2,4e429cead1848a298432a0acad014c9d,39.78125
3,80bc3ccafcc51b66303c2c263aa38486,40.40625
4,8f49844c382931444e68dffbe20228f4,40.00000
5,b03f7041962238a7c9d6537e22f9b017,39.96875
6,c978013571258ed6d4637f6e8cc9d6a3,39.31250
7,e0de453c1bffc20c22b072b34b54e50f,40.15625
